### **0. Import the relevant packages**

In [1]:
import os

# Setup projects
SIMPLE_RAG_PROJECT = "simple-rag"
os.environ["PHOENIX_PROJECT_NAME"] = SIMPLE_RAG_PROJECT

In [18]:
import pandas as pd
import phoenix as px
import nest_asyncio
from llama_index.core import (
    ServiceContext, StorageContext, download_loader,
    load_index_from_storage, set_global_handler
)

from datasets import load_dataset
from dotenv import load_dotenv

In [3]:
from phoenix.evals import (
    HallucinationEvaluator, OpenAIModel, QAEvaluator,
    RelevanceEvaluator, run_evals
)
from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents
from phoenix.trace import DocumentEvaluations, SpanEvaluations
from tqdm import tqdm

import qdrant_client
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import PointStruct

nest_asyncio.apply()  # needed for concurrent evals in notebook environments
pd.set_option("display.max_colwidth", 1000)

### **1. Launch Phoenix**
You can run Phoenix in the background to collect trace data emitted by any LlamaIndex application that has been instrumented with the OpenInferenceTraceCallbackHandler. Phoenix supports LlamaIndex's one-click observability which will automatically instrument your LlamaIndex application! You can consult our integration guide for a more detailed explanation of how to instrument your LlamaIndex application.

Launch Phoenix and follow the instructions in the cell output to open the Phoenix UI (the UI should be empty because we have yet to run the LlamaIndex application).

In [4]:
session = px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


Be sure to enable phoenix as your global handler for tracing!

In [5]:
set_global_handler("arize_phoenix")

In [6]:
# We can view the above data in the UI
px.active_session().view()

📺 Opening a view to the Phoenix app. The app is running at http://localhost:6006/


### **2. Run Your Query Engine and View Your Traces in Phoenix**

We've compiled a list of commonly asked questions about Qdrant. Let's download the sample queries and take a look.

In [8]:
## Loading the Eval dataset
qdrant_qa = load_dataset("atitaarora/qdrant_docs_qna_ragas", split="train")
qdrant_qa_question = qdrant_qa.select_columns(['question'])

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
qdrant_qa_question['question'][:10]

['What is the purpose of oversampling in Qdrant search process?',
 'How does Qdrant address the search accuracy problem in comparison to other search engines using HNSW?',
 'What is the difference between regular and neural search?',
 'How can I use Qdrant as a vector store in Langchain Go?',
 'How did Dust leverage compression features in Qdrant to manage the balance between storing vectors on disk and keeping quantized vectors in RAM effectively?',
 'Why do we still need keyword search?',
 'What principles did Qdrant follow while designing benchmarks for vector search engines?',
 'What models does Qdrant support for embedding generation?',
 'How can you parallelize the upload of a large dataset using shards in Qdrant?',
 'What is the significance of maximizing the distance between all points in the response when utilizing vector similarity for diversity search?']

### *This example uses a QdrantVectorStore and uses the previously generated collection to work fully connected with Qdrant but you can use whatever LlamaIndex application you like.*

In [20]:
load_dotenv()

False

In [14]:
COLLECTION_NAME = "qdrant_docs_arize_dense"

In [15]:
##Uncomment to initialise qdrant client in memory
#client = qdrant_client.QdrantClient(
#    location=":memory:",
#)

##Uncomment below to connect to Qdrant Cloud
client = QdrantClient(
    os.environ.get("QDRANT_URL"), 
    api_key=os.environ.get("QDRANT_API_KEY"),
)

## Uncomment below to connect to local Qdrant
#client = qdrant_client.QdrantClient("http://localhost:6333")

In [16]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='semantic search'), CollectionDescription(name='rag-knowledge-base-MiniLM'), CollectionDescription(name='qdrant-docs'), CollectionDescription(name='knowledge-base'), CollectionDescription(name='med-db')])

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from llama_index.core.indices.vector_store import VectorIndexRetriever

vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION_NAME)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = load_index_from_storage(storage_context)

retriever = VectorIndexRetriever(
    index=index,
    vector_store_query_mode=VectorStoreQueryMode.DEFAULT,
    similarity_top_k=5
)

In [ ]:
query_engine = index.as_query_engine()
for query in tqdm(qdrant_qa_question['question'][:10]):
    try:
      query_engine.query(query)
      #retriever.retrieve(query)
    except Exception as e:
      pass